In [27]:
import sys
import os
from datetime import datetime
from google.oauth2 import service_account
from googleapiclient import discovery
from googleapiclient.http import MediaFileUpload
from datetime import datetime
from utils.df_handle import *
import pandas as pd
import numpy as np
import time
import pyodbc
print(os.path.dirname(sys.executable))

d:\data_sale\venv\Scripts


In [28]:
server = '115.165.164.234'
driver = 'SQL Server'
db1 = 'PhaNam_eSales_PRO'
tcon = 'no'
uname = 'duyvq'
pword = '123VanQuangDuy'
# cnxn = pyodbc.connect(driver='{ODBC Driver 17 for SQL Server}', 
#                       host=server, database=db1, trusted_connection=tcon,
#                       user=uname, password=pword)
# cursor = cnxn.cursor()
# df1 = pd.read_sql(query, cnxn, parse_dates={"DateOfOrder": {"dayfirst": True}})

In [29]:
datenow = datetime.now().strftime("%Y%m%d")
param_1 = f"'20161130'"
param_2 = f"'20210930'"
param_3 = f"'{datenow}'"

In [30]:
cnxn = pyodbc.connect(driver='{ODBC Driver 17 for SQL Server}', 
                      host=server, database=db1, trusted_connection=tcon,
                      user=uname, password=pword)
query = f"EXEC [pr_AR_TrackingDebtConfirm]  @Fromdate={param_1}, @Todate={param_2}"
start=time.time()
df1 = pd.read_sql(query, cnxn, parse_dates={"DateOfOrder": {"dayfirst": True}, "DueDate": {"dayfirst": True}})
df1.to_csv('doanhthutienmat_data1.csv', index = False)
end=time.time()
duration=end-start
print(duration)
cnxn.close()

126.51818561553955


In [53]:
df1 = pd.read_csv('doanhthutienmat_data1.csv', dayfirst=True, parse_dates=['DateOfOrder', 'DueDate'])

In [54]:
print(df1['OrdAmtRelease'].sum())

185727532348.0


In [55]:
df_vptt = pd.read_csv('pvtt.csv')
df1 = df1.merge(df_vptt, how = 'left', left_on='State', right_on='tinh',suffixes=('_left', '_right'), validate="m:1")

In [56]:
df1['vptt'].value_counts(dropna=False)

TRẠM             114178
VP TRỰC THUỘC     78107
Name: vptt, dtype: int64

In [57]:
df1['State'][df1['vptt'].isna()]

Series([], Name: State, dtype: object)

In [58]:
df1[['State','vptt']].isna()

,State,vptt
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
192280,False,False
192281,False,False
192282,False,False
192283,False,False


In [59]:
df1[['DateOfOrder', 'DueDate']].dtypes

DateOfOrder    datetime64[ns]
DueDate        datetime64[ns]
dtype: object

In [60]:
df1.SupName.value_counts(dropna=False)

Nguyễn Hoàng Nguyên    23789
Nguyễn Phước Hòa       20400
Lâm Mạnh Quyền         18611
Nguyễn Hoàng Mạnh      18400
Nguyễn Văn Tiến        17417
Nguyễn Tùng Lâm        15787
Trương Tới             15645
Nguyễn Minh Thảo       15166
Vũ Đình Thắng          13225
Nguyễn Thái Bảo        11619
Phan Việt Khải         10756
Lê Tấn Hải              5088
Nguyễn Chí Tâm          3979
Nguyễn Xuân Kiệt        1861
NaN                      355
Nguyễn Thị Hằng          113
Hoàng Trung Thành         28
Nguyễn Thành Hiệp         28
Lâm Văn Cảnh              13
Lương Trịnh Thắng          5
Name: SupName, dtype: int64

In [61]:
groupbylist = [
    'OrderNbr', 
    'BranchID', 
    'Position', 
    'SlsperID', 
    'SupName', 
    'ASMName', 
    'RSMName', 
    'DateOfOrder', 
    'DueDate','SlsperName', 
    'Terms', 
    'PaymentsForm',
    'TermsType',
    'Territory', 
    'State', 
    'vptt', 
    'DeliveryUnit',
    'SubChannel'
    ]

In [62]:
aggregate_dict = {
'OpeiningOrderAmt':np.sum,
'OrdAmtRelease':np.sum,
'DeliveredOrderAmt':np.sum,
'ReturnOrdAmt': np.sum,
'DebConfirmAmt': np.sum,
'DebConfirmAmtRelease': np.sum,
#DonHang8
'CountOpeningOrder':np.sum,
'CountOrdRelease': np.sum,
'DeliveredOrder':np.sum,
'CountReturnOrd': np.sum,
'CountDebtConfirm': np.sum,
'CountDebtConfirmRelease':np.sum
}

In [63]:
df2 = df1.groupby(groupbylist, dropna=False).aggregate(aggregate_dict).reset_index()

In [64]:
print(df2['OrdAmtRelease'].sum()==185823616294)

False


In [65]:
df2['DateOfOrder'].nsmallest(5)

0      2017-01-18
1      2017-01-18
2      2017-01-18
67     2017-02-23
3194   2017-05-25
Name: DateOfOrder, dtype: datetime64[ns]

In [66]:
df2.columns

Index(['OrderNbr', 'BranchID', 'Position', 'SlsperID', 'SupName', 'ASMName',
       'RSMName', 'DateOfOrder', 'DueDate', 'SlsperName', 'Terms',
       'PaymentsForm', 'TermsType', 'Territory', 'State', 'vptt',
       'DeliveryUnit', 'SubChannel', 'OpeiningOrderAmt', 'OrdAmtRelease',
       'DeliveredOrderAmt', 'ReturnOrdAmt', 'DebConfirmAmt',
       'DebConfirmAmtRelease', 'CountOpeningOrder', 'CountOrdRelease',
       'DeliveredOrder', 'CountReturnOrd', 'CountDebtConfirm',
       'CountDebtConfirmRelease'],
      dtype='object')

In [67]:
rename_dict ={
# Doanh So
'OpeiningOrderAmt': 'tiennodauky',
'OrdAmtRelease':'tienchotso',
'DeliveredOrderAmt':'tiengiaothanhcong',
'ReturnOrdAmt':'tienhuydon',
'DebConfirmAmt':'tienlenbangke',
'DebConfirmAmtRelease':'tienthuquyxacnhan',
# Don Hang
'CountOpeningOrder': 'dondauky',
'CountOrdRelease':'donchotso',
'DeliveredOrder':'dongiaothanhcong',
'CountReturnOrd':'donhuy',
'CountDebtConfirm':'donlenbangke',
'CountDebtConfirmRelease':'donthuquyxacnhan'
}
df2.rename(columns=rename_dict, inplace=True)

In [79]:
df2b = df2[df2['tiennodauky'] > 0]

In [80]:
df2b['DateOfOrder'].nlargest(10)

1044   2021-04-29
1065   2021-04-29
1112   2021-04-29
1147   2021-04-29
1148   2021-04-29
1214   2021-04-29
1265   2021-04-29
1413   2021-04-29
1428   2021-04-29
1473   2021-04-29
Name: DateOfOrder, dtype: datetime64[ns]

In [69]:
df2['tiennocongty'] = df2['tiennodauky'] + df2['tienchotso'].values - df2['tienhuydon'].values - df2['tienthuquyxacnhan'].values
# df2['tiennocongty'] = np.where(df2['chitietnocongty']>0, df2['chitietnocongty'].values, 0)
# df2['chitietdonnocongty'] = df2['dondauky'] + df2['donchotso'] - df2['donthuquyxacnhan'] - df2['donhuy']
# df2['donnocongty_old'] = df2.apply(lambda x: x['chitietdonnocongty'] if x['chitietdonnocongty']>0 else 0, axis=1)
df2['donnocongty'] = np.where(df2['tiennocongty'].values > 0, 1, 0)

df2['donchuagiao'] = df2['donchotso'] - df2['dongiaothanhcong'] - df2['donhuy']
df2['tiendonchuagiao'] = df2['tienchotso'] - df2['tiengiaothanhcong'] - df2['tienhuydon']

date_now_at_midnight = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
df2['OverDueDate'] = np.where(df2['DueDate'].values >= np.datetime64(date_now_at_midnight), True, False)
# df2['OverDueDate'].value_counts()
df2['nodenhan'] = np.where(df2['OverDueDate'].values, df2['tiennocongty'].values, 0)
df2['nochuadenhan'] = np.where(df2['OverDueDate'].values != True, df2['tiennocongty'].values, 0)

In [72]:
un(df2, 'TermsType')

array(['Cho nợ', 'Thanh Toán Ngay'], dtype=object)

In [75]:
df2a = df_filter(df2, Position='LOG', TermsType='Thanh Toán Ngay', PaymentsForm='CK')

In [78]:
print(df2a['tienchotso'].sum())
print(df2a['tienthuquyxacnhan'].sum())
print(df2a['tiengiaothanhcong'].sum())
print(df2a['tienhuydon'].sum())

187719399.0
187719399.0
160840399.0
0.0


In [48]:
for l in df2.columns:
    print(l)

OrderNbr
BranchID
Position
SlsperID
SupName
ASMName
RSMName
DateOfOrder
DueDate
SlsperName
Terms
PaymentsForm
TermsType
Territory
State
vptt
DeliveryUnit
SubChannel
tiennodauky
tienchotso
tiengiaothanhcong
tienhuydon
tienlenbangke
tienthuquyxacnhan
dondauky
donchotso
dongiaothanhcong
donhuy
donlenbangke
donthuquyxacnhan
tiennocongty
donnocongty
donchuagiao
tiendonchuagiao
OverDueDate
nodenhan
nochuadenhan


In [49]:
col_list = [
    'OrderN',
    'BranchID',
    'Position',
    'SlsperID',
    'Tên Quản Lý',
    'Tên Quản Lý Khu Vực',
    'Tên Quản Lý Vùng',
    'Ngày Tạo Đơn',
    'Ngày Đến Hạn',
    #9
    'MDS',
    'Terms',
    'PaymentsForm',
    'Hạn Thanh Toán',
    'Khu Vực',
    'Tỉnh',
    'Phân Vùng Dịch Vụ',
    'Đơn Vị Giao Hàng',
    'Kênh Phụ',
    #15
    'Tiền Nợ Đầu Kỳ',
    'Tiền Chốt Sổ',
    'Tiền Giao Thành Công',
    'Tiền Hủy Đơn',
    'Tiền Lên Bảng Kê',
    'Tiền Thủ Quỹ Xác Nhận',
    #21
    'Đơn Nợ Đầu Kỳ',
    'Đơn Chốt Sổ',
    'Đơn Giao Thành Công',
    'Don Huy',
    'Đơn Lên Bảng Kê',
    'Đơn Thủ Quỹ Xác Nhận',
    #27
    'Dư Nợ Cuối Kỳ',
    'Đơn Nợ Công Ty',
    'Đơn Chưa Giao',
    'Tiền Đơn Chưa Giao',
    'OverDueDate',
    'Nợ Đến Hạn',
    'Nợ Chưa Đến Hạn'
# tiennocongty
# donnocongty
# donchuagiao
# tiendonchuagiao
# OverDueDate
# nodenhan
# nochuadenhan
]

In [50]:
df2.columns

Index(['OrderNbr', 'BranchID', 'Position', 'SlsperID', 'SupName', 'ASMName',
       'RSMName', 'DateOfOrder', 'DueDate', 'SlsperName', 'Terms',
       'PaymentsForm', 'TermsType', 'Territory', 'State', 'vptt',
       'DeliveryUnit', 'SubChannel', 'tiennodauky', 'tienchotso',
       'tiengiaothanhcong', 'tienhuydon', 'tienlenbangke', 'tienthuquyxacnhan',
       'dondauky', 'donchotso', 'dongiaothanhcong', 'donhuy', 'donlenbangke',
       'donthuquyxacnhan', 'tiennocongty', 'donnocongty', 'donchuagiao',
       'tiendonchuagiao', 'OverDueDate', 'nodenhan', 'nochuadenhan'],
      dtype='object')

In [51]:
df2.columns = col_list
df2.to_csv('doanhthutienmat_data.csv', index=False)

In [ ]:
# Google Sheet connection
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.file']
jsonfile = 'D:/data_sale/datateam1599968716114-6f9f144b4262.json'
credentials = service_account.Credentials.from_service_account_file(jsonfile, scopes=scopes)
service = discovery.build('drive', 'v3', credentials=credentials)
#spreadsheets_id = '1qEwviiJcAtvWCLvc-5AXOgidaeHMsi2RWWIKiUehabI'
folder_id = '12iN7KEKpgT4QhIfaLNB5shQ4g5x-VUoY'
file_name = 'doanhthutienmat_data.csv'
file_type = 'text/csv'
file_body={'name':file_name,'addParents':[folder_id]}
query = f"parents = '{folder_id}'"
files = service.files().list(q=query).execute()
fileid  = files['files'][0]['id']

media = MediaFileUpload('{0}'.format(file_name), mimetype=file_type)
updated_file = service.files().update(fileId=fileid, body=file_body, media_body=media)
updated_file.execute()